In [1]:
import os
import sys
import numpy as np
from skimage import io
import itk
from scipy import ndimage
from itkwidgets import compare, checkerboard, view

In [2]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/DK52/preps/CH1'
REGDATA = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'

In [30]:
filename = 'allen_50um_sagittal.tif'
fixedFilepath = os.path.join(REGDATA, filename)
fixed_image = itk.imread(fixedFilepath, itk.F)

In [12]:
movingFilepath = os.path.join(DATA, 'aligned_volume.256.tif')
moving_image = itk.imread(movingFilepath, itk.F)

In [27]:
parameter_object = itk.ParameterObject.New()
trans_parameter_map = parameter_object.GetDefaultParameterMap('translation')
rigid_parameter_map = parameter_object.GetDefaultParameterMap('rigid')
affine_parameter_map = parameter_object.GetDefaultParameterMap('affine')
bspline_parameter_map = parameter_object.GetDefaultParameterMap("bspline")
parameter_object.AddParameterMap(trans_parameter_map)
parameter_object.AddParameterMap(rigid_parameter_map)
parameter_object.AddParameterMap(affine_parameter_map)
parameter_object.AddParameterMap(bspline_parameter_map)

In [28]:
%%time
elastix_object = itk.ElastixRegistrationMethod.New(fixed_image, moving_image)
elastix_object.SetParameterObject(parameter_object)
# Set additional options
elastix_object.SetLogToConsole(False)
#elastix_object.SetNumberOfThreads(2)
# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()
resultImage = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()

CPU times: user 1min 45s, sys: 2.27 s, total: 1min 48s
Wall time: 17.9 s


In [26]:
type(resultImage)
result_transform_parameters

<itk.elxParameterObjectPython.elastixParameterObject; proxy of <Swig Object of type 'elastixParameterObject *' at 0x7f7e485c6730> >

In [ ]:
# 1- blue is fixed (Allen), 2 red is moving (DKXX)
#view(resultImage)

In [29]:
Dimension = 3
InputPixelType = itk.F
OutputPixelType = itk.UC
InputImageType = itk.Image[InputPixelType, Dimension]
OutputImageType = itk.Image[OutputPixelType, Dimension]
castImageFilter = itk.CastImageFilter[InputImageType, OutputImageType].New()
outpath = os.path.join(REGDATA, 'allen_test_affine_bpsline.tif')
#writer = itk.ImageFileWriter[OutputImageType].New()
#writer.SetFileName(outpath)
#writer.SetInput(castImageFilter.GetOutput())
#writer.Update()

itk.imwrite(resultImage, outpath)